In [1]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings('ignore')

#Change Dates here
Start_Date = 20250201
End_Date = 20250209


LY_Start_Date = Start_Date - 10000
LY_End_Date = End_Date - 10000
#sales data
data = pd.read_csv('../Files/DSR/PF SALES 10 FEB 2025.csv')
LY_data = pd.read_csv('../Files/DSR/2024 SALES/FEB 2024 PF BAGWise SALES ALL BRANDS.csv')
#Visibility data
v_data = pd.read_csv('../Files/DSR/PF VISIBILITY 10 FEB 2025.CSV')
LY_v_data = pd.read_csv('../Files/DSR/2024 VISIBILITY/PF DAILY VISIBILITY FEB 2024.csv')


data['master_brand'].replace('ADIDAS','Adidas',inplace=True)
LY_data['master_brand'].replace('ADIDAS','Adidas',inplace=True)
data['master_brand'].replace('ADIDAS Originals','Adidas Originals',inplace=True)
LY_data['master_brand'].replace('ADIDAS Originals','Adidas Originals',inplace=True)
data['master_brand'].replace('Cultsport','CULT',inplace=True)
LY_data['master_brand'].replace('Cultsport','CULT',inplace=True)


data = data[ ( data['order_created_date'] >= Start_Date ) & ( data['order_created_date'] <= End_Date ) 
           & ( (data['business_unit'] == "Women's Sports Apparel") | (data['business_unit'] == "Men's Sports Apparel")) 
           & (data['master_category'] == "Apparel") & ((data['gender'] == "Women") | (data['gender'] == "Men"))  ]
LY_data = LY_data[ ( LY_data['order_created_date'] >= LY_Start_Date ) & ( LY_data['order_created_date'] <= LY_End_Date ) 
                 & ( (LY_data['business_unit'] == "Women's Sports Apparel") | (LY_data['business_unit'] == "Men's Sports Apparel")) 
                 & (LY_data['master_category'] == "Apparel") & ((LY_data['gender'] == "Women") | (LY_data['gender'] == "Men")) ]


v_data['master_brand'].replace('ADIDAS','Adidas',inplace=True)
LY_v_data['master_brand'].replace('ADIDAS','Adidas',inplace=True)
v_data['master_brand'].replace('ADIDAS Originals','Adidas Originals',inplace=True)
LY_v_data['master_brand'].replace('ADIDAS Originals','Adidas Originals',inplace=True)
v_data['master_brand'].replace('Cultsport','CULT',inplace=True)
LY_v_data['master_brand'].replace('Cultsport','CULT',inplace=True)

v_data = v_data[ (v_data['load_date'] >= Start_Date) & (v_data['load_date'] <= End_Date) & ( (v_data['business_unit'] == "Women's Sports Apparel") | 
                 (v_data['business_unit'] == "Men's Sports Apparel")) & (v_data['master_category'] == "Apparel")  
                  & ((v_data['gender'] == "Women") | (v_data['gender'] == "Men"))   ]

LY_v_data = LY_v_data[ (LY_v_data['load_date'] >= LY_Start_Date) & (LY_v_data['load_date'] <= LY_End_Date) & 
                       ( (LY_v_data['business_unit'] == "Women's Sports Apparel") | 
                       (LY_v_data['business_unit'] == "Men's Sports Apparel")) & (LY_v_data['master_category'] == "Apparel")  
                       & ((LY_v_data['gender'] == "Women") | (LY_v_data['gender'] == "Men"))   ]

In [3]:


def compute_revenue(dump):
    if len(dump) == 0:
        return 0
    else:
        rev = round(((dump['revenue'].sum()) / 100000) ,4)
        return rev

def compute_ASP(dump):
    if len(dump) == 0:
        return '-'
    else:
        rev = dump['revenue'].sum()
        qty = dump['quantity'].sum()
        asp = round( (rev/qty) ,4)
        return asp

def compute_TD(dump):
    if len(dump) == 0:
        return '-'
    else:
        rev = dump['revenue'].sum()
        tmrp = dump['tot_mrp'].sum()
        discount = round( ((tmrp-rev)/(tmrp)) ,4)
        return discount

def compute_MSHARE(brand_dump,dump):
    if len(dump) == 0:
        return '-'
    else:
        share = round( (brand_dump['revenue'].sum() / dump['revenue'].sum()) , 4 )
        return share

def compute_visibility(dump):
    if len(dump) == 0:
        return '-'
    else:
        visi = round( (dump['list_view'].sum() / 10000000),2 )
        return visi

brands = list(set(data['master_brand'].unique()).union(LY_data['master_brand'].unique()))

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []

for brand in brands:
    dump = data[ (data['business_unit'] == "Men's Sports Apparel") & (data['master_brand'] == brand) ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['master_brand'] == brand) ]
    v_dump = v_data[ (v_data['business_unit'] == "Men's Sports Apparel") & (v_data['master_brand'] == brand) ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Men's Sports Apparel") & (LY_v_data['master_brand'] == brand) ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Men's Sports Apparel")  ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel")  ])
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)

data_MSA = {
    'MSA Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

MSA = pd.DataFrame(data_MSA).sort_values('TY REVENUE',ascending=False).set_index('MSA Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel")  ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA = {
    'WSA Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA = pd.DataFrame(data_WSA).sort_values('TY REVENUE',ascending=False).set_index('WSA Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Men's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Tshirts") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Tshirts") ]
    v_dump = v_data[ (v_data['business_unit'] == "Men's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Tshirts") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Men's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Tshirts") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Men's Sports Apparel") & (data['article_type'] == "Tshirts") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['article_type'] == "Tshirts")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_MSA_TSHIRTS = {
    'MSA Tshirts Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

MSA_TSHIRTS = pd.DataFrame(data_MSA_TSHIRTS).sort_values('TY REVENUE',ascending=False).set_index('MSA Tshirts Brand')

###########################################################################


REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Men's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Track Pants") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Track Pants") ]
    v_dump = v_data[ (v_data['business_unit'] == "Men's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Track Pants") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Men's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Track Pants") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Men's Sports Apparel") & (data['article_type'] == "Track Pants") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['article_type'] == "Track Pants")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_MSA_TRACKPANTS = {
    'MSA Track Pants Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

MSA_TRACKPANTS = pd.DataFrame(data_MSA_TRACKPANTS).sort_values('TY REVENUE',ascending=False).set_index('MSA Track Pants Brand')
###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Men's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Shorts") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Shorts") ]
    v_dump = v_data[ (v_data['business_unit'] == "Men's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Shorts") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Men's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Shorts") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Men's Sports Apparel") & (data['article_type'] == "Shorts") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Men's Sports Apparel") & (LY_data['article_type'] == "Shorts")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_MSA_SHORTS = {
    'MSA Shorts Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

MSA_SHORTS = pd.DataFrame(data_MSA_SHORTS ).sort_values('TY REVENUE',ascending=False).set_index('MSA Shorts Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Tshirts") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Tshirts") ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Tshirts") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Tshirts") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel") & (data['article_type'] == "Tshirts") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['article_type'] == "Tshirts")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA_TSHIRTS = {
    'WSA Tshirts Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA_TSHIRTS = pd.DataFrame(data_WSA_TSHIRTS).sort_values('TY REVENUE',ascending=False).set_index('WSA Tshirts Brand')

###########################################################################



REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Track Pants") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Track Pants") ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Track Pants") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Track Pants") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel") & (data['article_type'] == "Track Pants") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['article_type'] == "Track Pants")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA_TRACKPANTS = {
    'WSA Track Pants Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA_TRACKPANTS = pd.DataFrame(data_WSA_TRACKPANTS).sort_values('TY REVENUE',ascending=False).set_index('WSA Track Pants Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Tights") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Tights") ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Tights") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Tights") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel") & (data['article_type'] == "Tights") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['article_type'] == "Tights")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA_TIGHTS = {
    'WSA Tights Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA_TIGHTS = pd.DataFrame(data_WSA_TIGHTS).sort_values('TY REVENUE',ascending=False).set_index('WSA Tights Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Bra") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Bra") ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Bra") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Bra") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel") & (data['article_type'] == "Bra") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['article_type'] == "Bra")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA_BRA = {
    'WSA Bra Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA_BRA = pd.DataFrame(data_WSA_BRA).sort_values('TY REVENUE',ascending=False).set_index('WSA Bra Brand')

###########################################################################

REVENUE = []
LY_REVENUE = []
ASP = []
LY_ASP = []
TD = []
LY_TD = []
MSHARE = []
LY_MSHARE = []
VISIBILITY = []
LY_VISIBILITY = []


for brand in brands:
    dump = data[ (data['business_unit'] == "Women's Sports Apparel") & (data['master_brand'] == brand) & (data['article_type'] == "Jackets") ]
    ly_dump = LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['master_brand'] == brand) & (LY_data['article_type'] == "Jackets") ]
    v_dump = v_data[ (v_data['business_unit'] == "Women's Sports Apparel") & (v_data['master_brand'] == brand) & (v_data['article_type'] == "Jackets") ]
    ly_v_dump = LY_v_data[ (LY_v_data['business_unit'] == "Women's Sports Apparel") & (LY_v_data['master_brand'] == brand) & (LY_v_data['article_type'] == "Jackets") ]
    rev = compute_revenue( dump )
    ly_rev = compute_revenue( ly_dump )
    asp = compute_ASP( dump )
    ly_asp = compute_ASP( ly_dump )
    disc = compute_TD( dump )
    ly_disc = compute_TD( ly_dump )
    share = compute_MSHARE(dump,data[ (data['business_unit'] == "Women's Sports Apparel") & (data['article_type'] == "Jackets") ]) 
    ly_share = compute_MSHARE(ly_dump,LY_data[ (LY_data['business_unit'] == "Women's Sports Apparel") & (LY_data['article_type'] == "Jackets")  ])
    REVENUE.append(rev)
    LY_REVENUE.append(ly_rev)
    ASP.append(asp)
    LY_ASP.append(ly_asp)
    TD.append(disc)
    LY_TD.append(ly_disc)
    MSHARE.append(share)
    LY_MSHARE.append(ly_share)
    visi = compute_visibility(v_dump)
    ly_visi = compute_visibility(ly_v_dump)
    VISIBILITY.append(visi)
    LY_VISIBILITY.append(ly_visi)


data_WSA_JACKETS = {
    'WSA Jackets Brand' : brands,
    'LY REVENUE' : LY_REVENUE,
    'LY AVG ASP' : LY_ASP,
    'LY TD%' : LY_TD,
    'LY M SHARE%' : LY_MSHARE,
    'LY VISIBILITY' : LY_VISIBILITY,
    'TY REVENUE' : REVENUE,
    'TY AVG ASP' : ASP,
    'TY TD%' : TD,
    'TY M SHARE%' : MSHARE,
    'VISIBILITY' : VISIBILITY,
    
}    

WSA_JACKETS = pd.DataFrame(data_WSA_JACKETS).sort_values('TY REVENUE',ascending=False).set_index('WSA Jackets Brand')

###########################################################################


with pd.ExcelWriter("COMPETITION-TEMP.xlsx",engine='openpyxl') as writer:
        MSA.to_excel(writer, sheet_name='Sheet1')
        WSA.to_excel(writer, startcol=12,startrow=0,header=True,index=True)
        MSA_TSHIRTS.to_excel(writer, startcol=24,startrow=0,header=True,index=True)
        MSA_TRACKPANTS.to_excel(writer, startcol=36,startrow=0,header=True,index=True)
        MSA_SHORTS.to_excel(writer, startcol=48,startrow=0,header=True,index=True)
        WSA_TSHIRTS.to_excel(writer,startcol=60,startrow=0,header=True,index=True)
        WSA_TRACKPANTS.to_excel(writer,startcol=72,startrow=0,header=True,index=True)
        WSA_TIGHTS.to_excel(writer,startcol=84,startrow=0,header=True,index=True)
        WSA_BRA.to_excel(writer,startcol=96,startrow=0,header=True,index=True)
        WSA_JACKETS.to_excel(writer,startcol=108,startrow=0,header=True,index=True)